In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky 
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from scipy import constants
from scipy import stats as sci_stats  
import nawrapper as nw
import math
import csv 

path = "/home/kw4043/project/Planck_cib_maps/"
path_beam = "/home/kw4043/project/Planck_cib_maps/beams/BeamWf_HFI_R3/"

path_HI_857_40 = "/home/kw4043/project/Planck_cib_maps/HI_maps/4.0e+20_gp40_857/"
path_HI_545_40 = "/home/kw4043/project/Planck_cib_maps/HI_maps/4.0e+20_gp40_545/"
path_HI_353_40 = "/home/kw4043/project/Planck_cib_maps/HI_maps/4.0e+20_gp40_353/"

##I'm gonna be using path_beam for the mcm's as well to separate from the previous notebooks

def eshow(x,**kwargs): 
    ''' Define a function to help us plot the maps neatly '''
    plots = enplot.get_plots(x, **kwargs)
    enplot.show(plots, method = "ipython")
    
def shrink_mask(mask, r):
    """Shrink the True part of boolean mask "mask" by a distance of r radians"""
    return mask.distance_transform(rmax=r) >= r

def coth(x):
    "function returns value for hyperbolic cotangent"
    return 1/np.tanh(x)

In [ ]:
#Creating the bins then converting our maps into namap_car objects
bins = nw.create_binning(lmax=2000, lmin=2,
                         widths=[15]*2 + [20]*3 + [25]*4 + [75]*2 + 1*[100] + 2*[150] + 1*[250] + 2*[400],
                         weight_function=(lambda ell : ell**2))
#Our box for maps
box = [[np.pi/2, -np.pi/2],[-np.pi, np.pi]]
imap = curvedsky.make_projectable_map_by_pos(box, lmax=2000, dims=(), oversample=2.0, dtype=float, verbose=False)

bl_y = hp.gauss_beam(np.radians(10/60), lmax = 2000)

### Loading in the beam files for original planck maps

In [ ]:
from astropy.io import fits
beam_test_fname_353 = path_beam + 'Bl_T_R3.01_fullsky_353px353p.fits'
beam_test_fname_545 = path_beam + 'Bl_T_R3.01_fullsky_545x545.fits'
beam_test_fname_857 = path_beam + 'Bl_T_R3.01_fullsky_857x857.fits'

In [ ]:
with fits.open(beam_test_fname_353) as hdul_353:
    b_ell_353 = hdul_353[1].data['TEMPERATURE']
        
with fits.open(beam_test_fname_545) as hdul_545:
    b_ell_545 = hdul_545[1].data['TEMPERATURE']
    
with fits.open(beam_test_fname_857) as hdul_857:
    b_ell_857 = hdul_857[1].data['TEMPERATURE']

In [ ]:
b_ell_353 = b_ell_353.copy()[0:2001]
b_ell_545 = b_ell_545.copy()[0:2001]
b_ell_857 = b_ell_857.copy()[0:2001]

In [ ]:
plt.plot(b_ell_545)

### Loading in the beam files for HI maps

In [ ]:
from astropy.io import ascii
beam_HI_857_fn_40 = path_HI_857_40 + 'windowfunctions.csv'
beam_HI_857_fn_40 = ascii.read(beam_HI_857_fn_40)
beam_HI_857_40 = beam_HI_857_fn_40['Wl_eff']
beam_HI_857_40 = np.array(beam_HI_857_40)
print(beam_HI_857_40)

### Reading in HI maps 

In [ ]:
#old planck files
pmap_fname_HI_857_40 = path_HI_857_40 + "cib_fullmission.hpx.fits"
HI_857_map_40 = hp.read_map(pmap_fname_HI_857_40, field=0)

In [ ]:
#reading in the mask they used
pmap_fname_HI_857_mask_40 = path_HI_857_40 + "mask_bool.hpx.fits"
HI_857_mask_40 = hp.read_map(pmap_fname_HI_857_mask_40, field=0)

In [ ]:
#changing the NaN values to the values in the nonmasked portions of the map
HI_857_map_with_mask_40 = HI_857_mask_40.copy()
trial = np.where(HI_857_map_with_mask_40 == 1)
HI_857_map_with_mask_40[trial] = HI_857_map_40[trial]

In [ ]:
HI_857_map_enmap_40 = reproject.enmap_from_healpix(HI_857_map_with_mask_40, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

In [ ]:
#old planck files
pmap_fname_HI_545_40 = path_HI_545_40 + "cib_fullmission.hpx.fits"
HI_545_map_40 = hp.read_map(pmap_fname_HI_545_40, field=0)

In [ ]:
#reading in the mask they used
pmap_fname_HI_545_mask_40 = path_HI_545_40 + "mask_bool.hpx.fits"
HI_545_mask_40 = hp.read_map(pmap_fname_HI_545_mask_40, field=0)

In [ ]:
#changing the NaN values to the values in the nonmasked portions of the map
HI_545_map_with_mask_40 = HI_545_mask_40.copy()
trial = np.where(HI_545_map_with_mask_40 == 1)
HI_545_map_with_mask_40[trial] = HI_545_map_40[trial]

In [ ]:
HI_545_map_enmap_40 = reproject.enmap_from_healpix(HI_545_map_with_mask_40, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

In [ ]:
#old planck files
pmap_fname_HI_353_40 = path_HI_353_40 + "cib_fullmission.hpx.fits"
HI_353_map_40 = hp.read_map(pmap_fname_HI_353_40, field=0)

In [ ]:
#reading in the mask they used
pmap_fname_HI_353_mask_40 = path_HI_353_40 + "mask_bool.hpx.fits"
HI_353_mask_40 = hp.read_map(pmap_fname_HI_353_mask_40, field=0)

In [ ]:
#changing the NaN values to the values in the nonmasked portions of the map
HI_353_map_with_mask_40 = HI_353_mask_40.copy()
trial = np.where(HI_353_map_with_mask_40 == 1)
HI_353_map_with_mask_40[trial] = HI_353_map_40[trial]

In [ ]:
HI_353_map_enmap_40 = reproject.enmap_from_healpix(HI_353_map_with_mask_40, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

### Reading in planck maps

In [ ]:
#this is our Y map
pmap_fname_MILCA = path + "COM_CompMap_Compton-SZMap-milca-ymaps_2048_R2.00.fits"
MILCA_Y_map = hp.read_map(pmap_fname_MILCA, field=0)

MILCA_Y_map_enmap = reproject.enmap_from_healpix(MILCA_Y_map, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

In [ ]:
pmap_fname_857 = path + "HFI_SkyMap_857_2048_R2.00_full.fits"
cib_map_857 = hp.read_map(pmap_fname_857, field=0)

cib_map_857_enmap = reproject.enmap_from_healpix(cib_map_857, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

In [ ]:
pmap_fname_545 = path + "HFI_SkyMap_545_2048_R2.00_full.fits"
cib_map_545 = hp.read_map(pmap_fname_545, field=0)

cib_map_545_enmap = reproject.enmap_from_healpix(cib_map_545, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

In [ ]:
pmap_fname_353 = path + "HFI_SkyMap_353_2048_R2.00_full.fits"
cib_map_353 = hp.read_map(pmap_fname_353, field=0)

cib_map_353_enmap = reproject.enmap_from_healpix(cib_map_353, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

In [ ]:
HI_857_mask_enmap_40 = reproject.enmap_from_healpix(HI_857_mask_40, imap.shape, imap.wcs,
                                  ncomp=1, unit=1, lmax=2000,rot='gal,equ')

### Unit Conversions for both healpix and enmap

In [ ]:
#Converting frequency maps from MJy/sr to Kelvin
cib_map_857_enmap = cib_map_857_enmap / 2.27 
HI_857_map_enmap_40 = HI_857_map_enmap_40 / 2.27 

cib_map_545_enmap = cib_map_545_enmap / 58.04
HI_545_map_enmap_40 = HI_545_map_enmap_40 / 58.04

HI_353_map_enmap_40 = HI_353_map_enmap_40 / 287.450

In [ ]:
#Converting frequency maps from Kelvin to Compton 
cib_map_857_enmap = cib_map_857_enmap * 0.0380
HI_857_map_enmap_40 = HI_857_map_enmap_40 * 0.0380

cib_map_545_enmap = cib_map_545_enmap * 0.06918
HI_545_map_enmap_40 = HI_545_map_enmap_40 * 0.06918

cib_map_353_enmap = cib_map_353_enmap * 0.161098
HI_353_map_enmap_40 = HI_353_map_enmap_40 * 0.161098

### Putting maps+mask into NaMaster

In [ ]:
MILCA_Y_object_HI_mask_40 = nw.namap_car(maps = (MILCA_Y_map_enmap[0], None, None), masks = HI_857_mask_enmap_40[0], beams = bl_y)

In [ ]:
cib_object_857_HI_map_40 = nw.namap_car(maps = (HI_857_map_enmap_40[0], None, None), masks = HI_857_mask_enmap_40[0], beams = beam_HI_857_40)
cib_object_545_HI_map_40 = nw.namap_car(maps = (HI_545_map_enmap_40[0], None, None), masks = HI_857_mask_enmap_40[0], beams = beam_HI_857_40)
cib_object_353_HI_map_40 = nw.namap_car(maps = (HI_353_map_enmap_40[0], None, None), masks = HI_857_mask_enmap_40[0], beams = beam_HI_857_40)

In [ ]:
cib_object_857_planck_40 = nw.namap_car(maps = (cib_map_857_enmap[0], None, None), masks = HI_857_mask_enmap_40[0], beams = b_ell_857)
cib_object_545_planck_40 = nw.namap_car(maps = (cib_map_545_enmap[0], None, None), masks = HI_857_mask_enmap_40[0], beams = b_ell_545)
cib_object_353_planck_40 = nw.namap_car(maps = (cib_map_353_enmap[0], None, None), masks = HI_857_mask_enmap_40[0], beams = b_ell_353)

In [ ]:
mc_planck_40 = nw.mode_coupling(MILCA_Y_object_HI_mask_40, cib_object_857_planck_40, bins = bins, mcm_dir = path_beam + 'mc_planck_40', overwrite = True)
mc_HI_40 = nw.mode_coupling(MILCA_Y_object_HI_mask_40, cib_object_857_HI_map_40, bins = bins, mcm_dir = path_beam + 'mc_HI_40', overwrite = True)

In [ ]:
mc_planck_40_auto = nw.mode_coupling(cib_object_857_planck_40, cib_object_857_planck_40, bins = bins, mcm_dir = path_beam + 'mc_planck_40_auto', overwrite = True)
mc_40_MILCA_auto = nw.mode_coupling(MILCA_Y_object_HI_mask_40, MILCA_Y_object_HI_mask_40, bins = bins, mcm_dir = path_beam + 'mc_40_MILCA_auto', overwrite = True)
mc_HI_40_auto = nw.mode_coupling(cib_object_857_HI_map_40, cib_object_857_HI_map_40, bins = bins, mcm_dir = path_beam + 'mc_HI_40_auto', overwrite = True)

In [ ]:
CB_353_planck_40 = nw.compute_spectra(MILCA_Y_object_HI_mask_40, cib_object_353_planck_40, mc=mc_planck_40, lmax = 2000, bins = bins)
CB_545_planck_40 = nw.compute_spectra(MILCA_Y_object_HI_mask_40, cib_object_545_planck_40, mc=mc_planck_40, lmax = 2000, bins = bins)
CB_857_planck_40 = nw.compute_spectra(MILCA_Y_object_HI_mask_40, cib_object_857_planck_40, mc=mc_planck_40, lmax = 2000, bins = bins)

In [ ]:
CB_353_HI_40 = nw.compute_spectra(MILCA_Y_object_HI_mask_40, cib_object_353_HI_map_40, mc=mc_HI_40, lmax = 2000, bins = bins)
CB_545_HI_40 = nw.compute_spectra(MILCA_Y_object_HI_mask_40, cib_object_545_HI_map_40, mc=mc_HI_40, lmax = 2000, bins = bins)
CB_857_HI_40 = nw.compute_spectra(MILCA_Y_object_HI_mask_40, cib_object_857_HI_map_40, mc=mc_HI_40, lmax = 2000, bins = bins)

In [ ]:
CB_353_planck_40_auto = nw.compute_spectra(cib_object_353_planck_40, cib_object_353_planck_40, mc=mc_planck_40_auto, lmax = 2000, bins = bins)
CB_545_planck_40_auto = nw.compute_spectra(cib_object_545_planck_40, cib_object_545_planck_40, mc=mc_planck_40_auto, lmax = 2000, bins = bins)
CB_857_planck_40_auto = nw.compute_spectra(cib_object_857_planck_40, cib_object_857_planck_40, mc=mc_planck_40_auto, lmax = 2000, bins = bins)

CB_353_HI_40_auto = nw.compute_spectra(cib_object_353_HI_map_40, cib_object_353_HI_map_40, mc=mc_HI_40_auto, lmax = 2000, bins = bins)
CB_545_HI_40_auto = nw.compute_spectra(cib_object_545_HI_map_40, cib_object_545_HI_map_40, mc=mc_HI_40_auto, lmax = 2000, bins = bins)
CB_857_HI_40_auto = nw.compute_spectra(cib_object_857_HI_map_40, cib_object_857_HI_map_40, mc=mc_HI_40_auto, lmax = 2000, bins = bins)

CB_40_MILCA_auto = nw.compute_spectra(MILCA_Y_object_HI_mask_40, MILCA_Y_object_HI_mask_40, mc=mc_40_MILCA_auto, lmax = 2000, bins = bins)

In [ ]:
f_sky_40 = 0.33826835950215656

bins_array = [15,15,20,20,20,25,25,25,25,75,75,100,150,150,250,400,400]

In [ ]:
yerr_857_planck_40 = (CB_857_planck_40['TT']*CB_857_planck_40['TT'] + CB_40_MILCA_auto['TT']*CB_857_planck_40_auto['TT'])/((2*CB_857_planck_40['ell']+1)*f_sky_40*bins_array)
yerr_545_planck_40 = (CB_545_planck_40['TT']*CB_545_planck_40['TT'] + CB_40_MILCA_auto['TT']*CB_545_planck_40_auto['TT'])/((2*CB_545_planck_40['ell']+1)*f_sky_40*bins_array)
yerr_353_planck_40 = (CB_353_planck_40['TT']*CB_353_planck_40['TT'] + CB_40_MILCA_auto['TT']*CB_353_planck_40_auto['TT'])/((2*CB_353_planck_40['ell']+1)*f_sky_40*bins_array)

In [ ]:
yerr_857_HI_40 = (CB_857_HI_40['TT']*CB_857_HI_40['TT'] + CB_40_MILCA_auto['TT']*CB_857_HI_40_auto['TT'])/((2*CB_857_HI_40['ell']+1)*f_sky_40*bins_array)
yerr_545_HI_40 = (CB_545_HI_40['TT']*CB_545_HI_40['TT'] + CB_40_MILCA_auto['TT']*CB_545_HI_40_auto['TT'])/((2*CB_545_HI_40['ell']+1)*f_sky_40*bins_array)
yerr_353_HI_40 = (CB_353_HI_40['TT']*CB_353_HI_40['TT'] + CB_40_MILCA_auto['TT']*CB_353_HI_40_auto['TT'])/((2*CB_353_HI_40['ell']+1)*f_sky_40*bins_array)

In [ ]:
ell_Planck = CB_857_HI_40['ell']
dell_Planck =  ell_Planck*(ell_Planck+1)/(2*np.pi)

yerr_857_planck_40 = (yerr_857_planck_40**0.5)*1e12*dell_Planck
yerr_545_planck_40 = (yerr_545_planck_40**0.5)*1e12*dell_Planck
yerr_353_planck_40 = (yerr_353_planck_40**0.5)*1e12*dell_Planck

yerr_857_HI_40 = (yerr_857_HI_40**0.5)*1e12*dell_Planck
yerr_545_HI_40 = (yerr_545_HI_40**0.5)*1e12*dell_Planck
yerr_353_HI_40 = (yerr_353_HI_40**0.5)*1e12*dell_Planck

In [ ]:
ell_Planck = CB_857_HI_40['ell']
dell_Planck =  ell_Planck*(ell_Planck+1)/(2*np.pi)
fig, axes = plt.subplots(1,1, figsize=(10,6), sharex=True)

#axes.scatter(ell_Planck,(CB_857_HI_40['TT']*dell_Planck*(1e12)), label = '4.0e HI Cleaned Map')
#axes.scatter(ell_Planck,(CB_857_planck_40['TT']*dell_Planck*(1e12)), label = '4.0e Planck Frequency Map')

axes.errorbar(ell_Planck,(CB_857_HI_40['TT']*dell_Planck*(1e12)), yerr= yerr_857_HI_40, label = '4.0e LIM-Cleaned maps',  ls = '', fmt = 'o', markersize = 3)
axes.errorbar(ell_Planck,(CB_857_planck_40['TT']*dell_Planck*(1e12)), yerr= yerr_857_planck_40, label = '4.0e Planck maps',  ls = '', fmt = 'o', markersize = 3)

axes.set_xlabel(r'$\ell$', fontsize = 14)
axes.set_ylabel(r'$\ell(\ell + 1)C_{\ell}^{yT}/(2\pi)$ [$10^{-12}$ sr]', fontsize = 14)
#axes.set_ylabel(r'$C_{\ell}^{yT}\ell(\ell + 1)/(2\pi)$', fontsize = 14)
#axes.set_ylim(1e-4,1e4)
axes.set_yscale('log')
axes.set_xscale('log')
axes.tick_params(labelright=True, right = True)
axes.set_title('MILCA Y map x 857 GHz Frequency Maps', fontsize = 14)
plt.legend()

In [ ]:
ell_Planck = CB_545_HI_40['ell']
dell_Planck =  ell_Planck*(ell_Planck+1)/(2*np.pi)
fig, axes = plt.subplots(1,1, figsize=(10,6), sharex=True)

#axes.scatter(ell_Planck,(CB_545_HI_40['TT']*dell_Planck*(1e12)), label = '4.0e HI maps')
#axes.scatter(ell_Planck,(CB_545_planck_40['TT']*dell_Planck*(1e12)), label = '4.0e Planck maps')

axes.errorbar(ell_Planck,(CB_545_HI_40['TT']*dell_Planck*(1e12)), yerr= yerr_545_HI_40, label = '4.0e HI maps',  ls = '', fmt = 'o', markersize = 3)
axes.errorbar(ell_Planck,(CB_545_planck_40['TT']*dell_Planck*(1e12)), yerr= yerr_545_planck_40, label = '4.0e Planck maps',  ls = '', fmt = 'o', markersize = 3)

axes.set_xlabel(r'$\ell$', fontsize = 14)
axes.set_ylabel(r'$\ell(\ell + 1)C_{\ell}^{yT}/(2\pi)$ [$10^{-12}$ sr]', fontsize = 14)
#axes.set_ylim(1e-4,1e4)
axes.set_yscale('log')
axes.set_xscale('log')
axes.tick_params(labelright=True, right = True)
axes.set_title('MILCA Y map x 545 GHz Frequency Maps', fontsize = 14)
plt.legend()

In [ ]:
ell_Planck = CB_353_HI_40['ell']
dell_Planck =  ell_Planck*(ell_Planck+1)/(2*np.pi)
fig, axes = plt.subplots(1,1, figsize=(10,6), sharex=True)

#axes.scatter(ell_Planck,(CB_353_HI_40['TT']*dell_Planck*(1e12)), label = '4.0e HI maps')
#axes.scatter(ell_Planck,(CB_353_planck_40['TT']*dell_Planck*(1e12)), label = '4.0e Planck maps')

axes.errorbar(ell_Planck,(CB_353_HI_40['TT']*dell_Planck*(1e12)), yerr= yerr_353_HI_40, label = '4.0e HI maps',  ls = '', fmt = 'o', markersize = 3)
axes.errorbar(ell_Planck,(CB_353_planck_40['TT']*dell_Planck*(1e12)), yerr= yerr_353_planck_40, label = '4.0e Planck maps',  ls = '', fmt = 'o', markersize = 3)

axes.set_xlabel(r'$\ell$', fontsize = 14)
axes.set_ylabel(r'$\ell(\ell + 1)C_{\ell}^{yT}/(2\pi)$ [$10^{-12}$ sr]', fontsize = 14)
#axes.set_ylim(1e-4,1e4)
axes.set_yscale('log')
axes.set_xscale('log')
axes.tick_params(labelright=True, right = True)
axes.set_title('MILCA Y map x 353 GHz Frequency Maps', fontsize = 14)
plt.legend()

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(10,6), sharex=True)
ell_Planck = CB_857_HI_40['ell']
dell_Planck =  ell_Planck*(ell_Planck+1)/(2*np.pi)

axes.scatter(ell_Planck,(CB_857_planck_40_auto['TT']*dell_Planck), label = '4.0e Planck map')
axes.scatter(ell_Planck,(CB_857_HI_40_auto['TT']*dell_Planck), label = '4.0e HI map')

axes.set_yscale('log')
axes.set_xscale('log')
axes.tick_params(labelright=True, right = True)
axes.set_title('Autospectra of Frequency Maps', fontsize = 14)
plt.legend()